In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import re
import numpy as np
import pandas as pd
import requests

!pip install datasets
from datasets import load_dataset, Dataset, concatenate_datasets
!pip install dart-fss
import dart_fss as dart
!pip install tabulate
from tabulate import tabulate

!pip install wikipedia
import wikipedia
!pip install wikipedia-api
import wikipediaapi
!pip install bs4
import bs4

!pip install finance-datareader
import FinanceDataReader as fdr
!pip install OpenDartReader
import OpenDartReader

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.7/145.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 61.5 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=828f1260171e5c3cc3f770d01d71c31267fab3a57ebaecb64de4e85a3b83aa52
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia

In [ ]:
# 사용자 에이전트 설정
wiki = wikipediaapi.Wikipedia(
    language='ko',
    extract_format=wikipediaapi.ExtractFormat.WIKI,
    user_agent='YourAppName/1.0 (yourname@example.com)'
)

In [ ]:
# Infobox 내용 파싱
def parse_infobox(infobox):
    info_dict = {}

    for row in infobox.find_all('tr'):
        header = row.find('th')
        value = row.find('td')

        if header and value:
            key = header.get_text(" ", strip=True)  # 헤더 텍스트
            val = value.get_text(" ", strip=True)  # 값 텍스트

            info_dict[key] = val

    return info_dict


In [ ]:
# TODO 1. 가져올 데이터 목록. FinanceDataReader -> KOSPI, KOSDAQ
# TODO 2. Wiki 정보 가져오기. Wikipedaapi
# TODO 3. 재무정보 기본, 가공 가져오기.

In [ ]:
# TODO 1. 가져올 데이터 목록. FinanceDataReader -> KOSPI, KOSDAQ
my_api = '142314425c28fd17490aceee7f6cca7d1e099a0a'
dart = OpenDartReader(my_api)

stock_list = fdr.StockListing('KRX') # 코스피, 코스닥, 코넥스 종목 모두 불러오기
stock_list = stock_list.loc[stock_list['Market'].isin(["KOSPI", "KOSDAQ"])] # Market 기준 필터링
stock_name_list = stock_list['Name'].tolist() # 선물과 같이 KOSPI에는 있으나 배당이 없을수밖에 없는 종목은 제외

krx = fdr.StockListing('KRX')

2670
Index(['Code', 'ISU_CD', 'Name', 'Market', 'Dept', 'Close', 'ChangeCode',
       'Changes', 'ChagesRatio', 'Open', 'High', 'Low', 'Volume', 'Amount',
       'Marcap', 'Stocks', 'MarketId'],
      dtype='object')
     Code        ISU_CD      Name Market Dept    Close ChangeCode  Changes  \
0  005930  KR7005930003      삼성전자  KOSPI         56200          2     -400   
1  000660  KR7000660001    SK하이닉스  KOSPI        202000          1     3800   
2  373220  KR7373220003  LG에너지솔루션  KOSPI        410000          1     3500   
3  207940  KR7207940008  삼성바이오로직스  KOSPI       1072000          1    31000   
4  005380  KR7005380001       현대차  KOSPI        224000          2    -4500   

   ChagesRatio     Open     High      Low    Volume        Amount  \
0        -0.71    56000    56900    55900  15772930  885621983200   
1         1.92   203000   206000   200500   3673296  747172810800   
2         0.86   419000   419000   407500    163507   67401316500   
3         2.98  1038000  1082000  1021

In [ ]:
df_krx = krx[['Code','ISU_CD','Name','Market']]#,'Sector','Industry','Region','Exchange']]

for idx, row in df_krx.iterrows():
  print('idx: ', idx, flush=True)
  corp_name = row['Name']
  try:
    page = wiki.page(corp_name)
    page_url = page.fullurl
    response = requests.get(page_url)
    soup = bs4.BeautifulSoup(response.text, 'html')
    infobox = soup.find('table', {'class': 'infobox'})
    # Infobox가 존재할 경우 파싱
    if infobox:
        infobox_data = parse_infobox(infobox)
        # 딕셔너리를 하나의 텍스트로 변환
        infobox_text = "\n".join([f"{key}: {value}" for key, value in infobox_data.items()])
        # print(infobox_text)
        print("Infobox Foune !!!!")
    else:
        print("Infobox Not found... ")

    df_krx.loc[idx, 'wiki_infobox'] = infobox_text
    df_krx.loc[idx, 'wiki_text'] = page.text
  except:
    pass

df_krx['wiki_text'].replace('', np.nan, inplace=True)
df_krx.dropna(subset=['wiki_text'], inplace=True)
print(df_krx.shape) # 2472
df_krx.to_parquet('/content/drive/My Drive/krx/data/df_krx_wiki.parquet')

idx:  0
Infobox Foune !!!!
idx:  1


<ipython-input-6-838270b42825>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_krx.loc[idx, 'wiki_infobox'] = infobox_text
<ipython-input-6-838270b42825>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_krx.loc[idx, 'wiki_text'] = page.text


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Infobox Foune !!!!
idx:  157
Infobox Foune !!!!
idx:  158
Infobox Foune !!!!
idx:  159
Infobox Foune !!!!
idx:  160
Infobox Foune !!!!
idx:  161
Infobox Foune !!!!
idx:  162
Infobox Foune !!!!
idx:  163
Infobox Foune !!!!
idx:  164
Infobox Foune !!!!
idx:  165
Infobox Foune !!!!
idx:  166
Infobox Foune !!!!
idx:  167
Infobox Foune !!!!
idx:  168
Infobox Foune !!!!
idx:  169
Infobox Foune !!!!
idx:  170
Infobox Foune !!!!
idx:  171
Infobox Foune !!!!
idx:  172
idx:  173
Infobox Foune !!!!
idx:  174
Infobox Not found... 
idx:  175
Infobox Not found... 
idx:  176
Infobox Foune !!!!
idx:  177
Infobox Foune !!!!
idx:  178
Infobox Foune !!!!
idx:  179
Infobox Foune !!!!
idx:  180
Infobox Foune !!!!
idx:  181
Infobox Foune !!!!
idx:  182
Infobox Foune !!!!
idx:  183
Infobox Foune !!!!
idx:  184
Infobox Foune !!!!
idx:  185
Infobox Foune !!!!
idx:  186
Infobox Foune !!!!
idx:  187
Infobox Not found... 
idx:  188
Infobox Foune !!!!
idx:  189
Infobox Foune !!!

In [ ]:
# Data-Preprocess
# TODO 0 : 기본 제공 Data: 모두 한국어!

# TODO 1 : [국내기업일반] SystemPrompt: Summarize Prompt:'wiki_text' into Response:'wiki_infobox'

# TODO 2 : [주식가격 예측] TheFinAI: sm -> Korean : Prompt(Text, Query) Response(Answer)
# TODO 3 : [재무회계] Dart: 재무상태표, PER같은 지표들 산출, 주식가격(quarterly) -> 주식가격이 어떻게 될지 예측해보고 그 이유를 생성

# TODO 4 : [금융일반] TheFinAI: CFA -> Korean


In [17]:
######## TODO 2. [주식가격 예측] TheFinAI: sm -> Korean : Prompt(Text, Query) Response(Answer)
# 다음 표는 주식 A에 대한 분석 결과입니다. 표를 바탕으로 향후 A의 주가가 상승할지 하락할지 예측하고 [증가] 혹은 [감소] 라고 답변하시요.
data_path_list = ['TheFinAI/flare-sm-bigdata','TheFinAI/flare-sm-acl','TheFinAI/flare-sm-cikm']

def create_dataset(data_path):
    ds_tr = load_dataset(data_path)['train']
    ds_te = load_dataset(data_path)['test']
    try:
      ds_va = load_dataset(data_path)['valid']
    except:
      ds_va = load_dataset(data_path)['validation']

    ds = concatenate_datasets([ds_tr, ds_te, ds_va])
    ds = ds.select_columns(['query','answer'])
    return ds

ds_list = []
for data_path in data_path_list:
  ds = create_dataset(data_path)
  ds_list.append(ds)

df_stock = concatenate_datasets(ds_list)
# print(df_stock.shape)

df_stock.to_parquet('/content/drive/My Drive/krx/data/df_stock.parquet')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)-00000-of-00001-4c97651cf23a4342.parquet:   0%|          | 0.00/9.78M [00:00<?, ?B/s]

(…)-00000-of-00001-7ba518568ea39642.parquet:   0%|          | 0.00/433k [00:00<?, ?B/s]

(…)-00000-of-00001-bcbe082671cc1fdb.parquet:   0%|          | 0.00/792k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4897 [00:00<?, ? examples/s]

...

(…)-00000-of-00001-b105ab56855808e4.parquet:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3396 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1143 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/431 [00:00<?, ? examples/s]

Dataset({
    features: ['query', 'answer'],
    num_rows: 39193
})


In [2]:
# df_stock = pd.read_parquet('/content/drive/My Drive/krx/data/df_stock.parquet', engine='pyarrow')

In [ ]:
data_path_list = ['amphora/krx-sample-instructions','Cartinoe5930/web_text_synthetic_dataset_50k','Cartinoe5930/raw_text_synthetic_dataset_50k']
def create_dataset(data_path):
    ds_tr = load_dataset(data_path)['train']
    ds_te = load_dataset(data_path)['test']

    ds = concatenate_datasets([ds_tr, ds_te])
    if data_path == 'amphora/krx-sample-instructions':
      ds = ds.select_columns(['prompt','response'])
    else:
      ds = ds.select_columns(['question','response'])
      ds = ds.rename_column('question','prompt')
    return ds


ds_list = []
for data_path in data_path_list:
  ds = create_dataset(data_path)
  ds_list.append(ds)

df_krx = concatenate_datasets(ds_list)

df_krx.to_parquet('/content/drive/My Drive/krx/data/df_krx.parquet')


In [5]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from transformers import pipeline

# # Step 2: Load the pre-trained model and tokenizer
# # model_name = "unsloth/Qwen2.5-7B-Instruct"
# # tokenizer = AutoTokenizer.from_pretrained(model_name)
# # model = AutoModelForCausalLM.from_pretrained(model_name)

# # # Step 3: Set up a pipeline for inference
# # generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# # Load the M2M100 model and tokenizer
# model_name = "facebook/m2m100_1.2B"  # or use "facebook/m2m100_1.2B" for better quality
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# # Set up the translation pipeline
# translator = pipeline("translation", model=model, tokenizer=tokenizer, src_lang="en", tgt_lang="ko")

# # Translate a sample text
# result = translator("The quick brown fox jumps over the lazy dog.")



# query_list = []
# answer_list = []
# for query, answer in zip(df_stock["query"], df_stock["answer"]):
#     # Translate the English text
#     query_ko = translator(query)[0]["translation_text"]
#     query_list.append(query_ko)

#     # Translate the English answer
#     answer_ko = translator(answer)[0]["translation_text"]
#     answer_list.append(answer_ko)

# print(query_list[0])

# # Output the first few translations
# # for i, item in enumerate(translated_texts[:5]):
#     # print(f"Original: {item['original_text']}\nTranslated: {item['translated_text']}\n")


tokenizer_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

Your input_length: 1081 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 1444 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


KeyboardInterrupt: 

In [ ]:
# 질문 생성용 prompt 포맷팅

# Trial 1
# qrys = []
# for inf, t in zip(df_krx['wiki_infobox'] ,df_krx['wiki_text']):
#     messages = [
#     # {"content": "Your job is creating quantitative finance questions in fluent Korean. You will be given a English QF question collected from the web. Restructure it to a test-like question, in formal Korean language. Return the question only.", "role": "system"},
#     {"content": "Your job is creating quantitative finance questions in fluent Korean. You will be given a infobox from wikipedia and text from wikipeda collected from the web. Restructure it to a test-like question, in formal Korean language. Return the question only.", "role": "system"},
#     {"content": inf + t, "role": "user"}]
#     qrys.append(messages)

# Trial 2
prompt_format = """Your job is creating quantitative finance questions in fluent Korean. You will be given an infobox from wikipedia and text from wikipeda collected from the web. Restructure it to a test-like question, in formal Korean language. Return the question only.

### wikipedia infobox:
{}

### wikipedia text:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["query"]
    outputs = examples["answer"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt_format.format(instruction, output) + EOS_TOKEN # 마지막에 eos token을 추가해줌으로써 모델이 출력을 끝마칠 수 있게 만들어 줍니다.
        texts.append(text)
    return { "formatted_text" : texts, }
pass

dataset = df_krx.map(formatting_prompts_func, batched = True)


In [ ]:
# Define text generation parameters
generation_args = {
    "max_length": 256,  # Set the maximum length of the generated text
    "num_return_sequences": 1,  # Number of generated outputs per input
    "do_sample": True,  # Enable sampling for more varied outputs
    "temperature": 0.7  # Control randomness of generation
}

df_krx_wiki = pd.DataFrame(dataset['formatted_text'])
print(df_krx_wiki.head())

for idx, row in df_krx_wiki.iterrows():
  # Generate text based on the prompt
  print(row)
  generated_text = generator(row['formatted_text'], max_length=256, num_return_sequences=1)
  dataset.loc[idx, 'question'] = generated_text


# dataset['question'] = dataset['formatted_text'].apply(lambda x: classifier(x, **generation_args)[0])#['generated_text'])

# Display the results
print(dataset.head())